<a href="https://colab.research.google.com/github/alexbatista18/Processamento-SSVEP/blob/main/Processamento_SSVEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import signal
import matplotlib.pyplot as plt
import plotly.io as pio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Criando dataframe e renomeando as colunas com o nome dos canais...
hz13 = pd.read_csv('/content/drive/MyDrive/ISD/Projeto FES/Coleta/Coleta SSVEP 13 06/SS Final/referenciado/red13')
hz9 = pd.read_csv('/content/drive/MyDrive/ISD/Projeto FES/Coleta/Coleta SSVEP 13 06/SS Final/referenciado/red9')
whz13 = pd.read_csv('/content/drive/MyDrive/ISD/Projeto FES/Coleta/Coleta SSVEP 13 06/SS Final/referenciado/bran13')
whz9 = pd.read_csv('/content/drive/MyDrive/ISD/Projeto FES/Coleta/Coleta SSVEP 13 06/SS Final/referenciado/bran9')
bl = pd.read_csv('/content/drive/MyDrive/ISD/Projeto FES/Coleta/Coleta SSVEP 13 06/SS Final/bl')

In [ ]:
hz13

,Time:500Hz,Epoch,Channel 1,Channel 2,Channel 3,Channel 4,Channel 5,Channel 6,Channel 7,Channel 8,...,Channel 18,Channel 19,Channel 20,Channel 21,Channel 22,Channel 23,Channel 24,Event Id,Event Date,Event Duration
0,0.000,0,-4922.637939,12164.520264,10691.668701,6577.996826,8562.469482,425.115967,-813.507080,3860.467529,...,-4661.553955,-896.710205,-9839.385986,5856.170654,502.459717,-1357.550049,-16622.149658,NaN,NaN,NaN
1,0.002,0,-4920.113118,12160.013835,10691.117350,6576.566569,8560.550944,428.080241,-814.546712,3858.353678,...,-4663.033040,-897.896322,-9837.544759,5857.963053,504.252116,-1356.050618,-16619.966634,NaN,NaN,NaN
2,0.004,0,-4918.731689,12159.197998,10691.571045,6578.094482,8561.590576,429.754639,-812.969971,3859.442139,...,-4662.677002,-898.419189,-9843.243408,5855.340576,501.239014,-1357.550049,-16625.079346,NaN,NaN,NaN
3,0.006,0,-4916.009521,12159.136963,10691.754150,6580.621338,8562.066650,429.302979,-811.859131,3863.238525,...,-4661.907959,-899.017334,-9846.624756,5852.935791,498.345947,-1359.710693,-16631.341553,NaN,NaN,NaN
4,0.008,0,-4916.560872,12160.489909,10693.595378,6579.288737,8563.077799,428.800456,-812.654622,3864.396159,...,-4660.555013,-899.178060,-9848.982747,5853.312174,497.941081,-1362.459310,-16633.943685,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60187,120.374,1880,-4884.798177,12121.647135,10741.471354,6537.418620,8628.141276,534.537760,-773.323568,3881.998698,...,-4833.186849,-891.292318,-10126.155599,5644.059245,166.471354,-1330.159505,-16505.403646,NaN,NaN,NaN
60188,120.376,1880,-4882.027181,12125.345866,10745.023600,6539.506022,8630.228678,536.820475,-770.796712,3884.672038,...,-4835.054525,-892.427572,-10131.978353,5639.652507,162.797038,-1334.077962,-16513.863118,NaN,NaN,NaN
60189,120.378,1880,-4886.857096,12126.180013,10741.219076,6536.092122,8628.572591,531.648763,-770.548503,3882.527669,...,-4833.341471,-888.663737,-10121.866862,5644.295247,165.681966,-1329.386393,-16504.337565,NaN,NaN,NaN
60190,120.380,1880,-4889.611816,12124.206543,10738.220215,6531.726074,8627.038574,526.989746,-772.326660,3878.747559,...,-4831.213379,-884.924316,-10108.703613,5650.280762,172.302246,-1324.475098,-16491.906738,NaN,NaN,NaN


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=hz13['Time:500Hz'], y=hz13['Channel 1'], name='FP1'))

In [ ]:
import plotly.graph_objects as go
from scipy import signal

fz = 500

def espectro_9(data):
    #Aplicando filtro passa banda de ordem 2 de 8 a 45 Hz.
    b1, a1 = signal.butter(2, [8, 45], btype='bandpass', fs=fz)
    o1_filt = signal.filtfilt(b1, a1, data['Channel 11'])
    o2_filt = signal.filtfilt(b1, a1, data['Channel 12'])

    #Aplicando a transformada de Fourier para visualizar o sinal no dominio da frequência.
    f, p = signal.welch(o1_filt, fz, nperseg=fz*4, noverlap=fz*0.75)
    f2, p2 = signal.welch(o2_filt, fz, nperseg=fz*4, noverlap=fz*0.75)

    #Plotando para visulizar as frequências.
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=f, y=p, name='O1'))
    fig.add_trace(go.Scatter(x=f2, y=p2, name='O2'))

    valores_marcados = [9, 18, 27, 39]
    valores_marcadosb = [13, 26, 39]

    #Função para marcar os pontos de intersse no plot (9, 18, 27 e 39 Hz).
    for valor in valores_marcados:
      fig.add_trace(go.Scatter(x=[valor], y=[p2[valor*4]],
                                mode='markers',
                                name=f'{valor} Hz',
                                marker=dict(symbol='circle', color='green', size=8)))
    for valor in valores_marcados:
      fig.add_trace(go.Scatter(x=[valor], y=[p[valor*4]],
                                mode='markers',
                                marker=dict(symbol='circle', color='green', size=8)))

    #Adicionando legendas ao plot.
    fig.update_layout(
        title="Red light - 9 Hz",
        xaxis=dict(
            range=[0, 45],
            tickfont=dict(
                size=16,
                family="Arial, sans-serif",
                color="black"
            )
        ),
        yaxis=dict(
            tickfont=dict(
                size=16,
                family="Arial, sans-serif",
                color="black"
            )
        ),
        legend=dict(
            font=dict(
                size=18,
                family="Arial, sans-serif",
                color="black"
            )
        ),
        title_font=dict(
            size=24,
            family="Arial, sans-serif",
            color="black"
        )
    )

    fig.update_layout(
        autosize=False,
        width=800,
        height=500
    )

    fig.show()


In [ ]:
import plotly.graph_objects as go
from scipy import signal

fz = 500

#Mesa coisa que a função 'espectro9', só que adaptada para mostrar os pontos de intersse no plot (13, 26, 39 Hz).
def espectro(data):
    b1, a1 = signal.butter(2, [8, 45], btype='bandpass', fs=fz)
    o1_filt = signal.filtfilt(b1, a1, data['Channel 11'])
    o2_filt = signal.filtfilt(b1, a1, data['Channel 12'])
    f, p = signal.welch(o1_filt, fz, nperseg=fz*4, noverlap=fz*0.75)
    f2, p2 = signal.welch(o2_filt, fz, nperseg=fz*4, noverlap=fz*0.75)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=f, y=p, name='O1'))
    fig.add_trace(go.Scatter(x=f2, y=p2, name='O2'))

    valores_marcados = [9, 18, 27, 39]
    valores_marcadosb = [13, 26, 39]


    for valor in valores_marcadosb:
      fig.add_trace(go.Scatter(x=[valor], y=[p2[valor*4]],
                                mode='markers',
                                name=f'{valor} Hz',
                                marker=dict(symbol='circle', color='green', size=8)))

    for valor in valores_marcadosb:
      fig.add_trace(go.Scatter(x=[valor], y=[p[valor*4]],
                                mode='markers',
                                marker=dict(symbol='circle', color='green', size=8)))

    fig.update_layout(
        title="Red light - 13 Hz",
        xaxis=dict(
            range=[0, 45],
            tickfont=dict(
                size=16,
                family="Arial, sans-serif",
                color="black"
            )
        ),
        yaxis=dict(
            tickfont=dict(
                size=16,
                family="Arial, sans-serif",
                color="black"
            )
        ),
        legend=dict(
            font=dict(
                size=18,
                family="Arial, sans-serif",
                color="black"
            )
        ),
        title_font=dict(
            size=24,
            family="Arial, sans-serif",
            color="black"
        )
    )

    fig.update_layout(
        autosize=False,
        width=800,
        height=500
    )

    fig.show()

In [ ]:
espectro(hz13)

In [ ]:
espectro_9(hz9)

In [ ]:
espectro(whz13)

In [ ]:
espectro_9(whz9)